<a href="https://colab.research.google.com/github/lucaskydelima/Optimization-with-Python-Pyomo/blob/main/S5Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
! pip install pyomo
! pip install -q cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [23]:
from pyomo.core.base.objective import maximize
from pyomo.core.base.set_types import NonNegativeIntegers

# Defining the model
model = pyo.ConcreteModel()

# Set
model.i = pyo.Set(initialize = ['Shirts', 'Shorts', 'Pants'])

# Params
model.P = pyo.Param(model.i, initialize={'Shirts':12, 'Shorts':8, 'Pants':15})
P = model.P

model.C = pyo.Param(model.i, initialize={'Shirts':6, 'Shorts':4, 'Pants':8})
C = model.C

model.L = pyo.Param(model.i, initialize={'Shirts':3, 'Shorts':2, 'Pants':6})
L = model.L

model.Cl = pyo.Param(model.i, initialize={'Shirts':4, 'Shorts':3, 'Pants':4})
Cl = model.Cl

model.R = pyo.Param(model.i, initialize={'Shirts':200, 'Shorts':150, 'Pants':100})
R = model.R
# BIG-M Value
model.M = pyo.Param(model.i, initialize={'Shirts':50, 'Shorts':75, 'Pants':40})
M = model.M

# Decision Variable
model.x = pyo.Var(model.i, within=pyo.Integers)
x = model.x

model.y = pyo.Var(model.i, within=pyo.Binary)
y = model.y

# Objective Function
def Objective_rule(model):
  return sum(P[i]*x[i] - C[i]*x[i] - R[i]*y[i] for i in model.i)

model.obj = pyo.Objective(rule = Objective_rule, sense = maximize)


# Constraints
def Constraint_1(model, i):
  return sum(L[i] * x[i] for i in model.i) <= 150

model.const1 = pyo.Constraint(model.i, rule = Constraint_1)

def Constraint_2(model, i):
  return sum(Cl[i] * x[i] for i in model.i) <= 160

model.const2 = pyo.Constraint(model.i, rule = Constraint_2)

def Constraint_3(model, i):
  return x[i] <= M[i]*y[i] 

model.const = pyo.Constraint(model.i, rule = Constraint_3)

# Solve
solver = pyo.SolverFactory('cplex_direct')
results = solver.solve(model)

print(results)
print('Objective Function = ', model.obj() )
for i in model.i:
  print('Number of', i, 'produced each week = ', x[i]())



Problem: 
- Name: 
  Lower bound: 75.0
  Upper bound: 75.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 3
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: -1
Solver: 
- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.003885030746459961
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Function =  75.0
Number of Shirts produced each week =  0.0
Number of Shorts produced each week =  0.0
Number of Pants produced each week =  25.0
